In [ ]:
install.packages('tidyverse')

also installing the dependencies 'highr', 'markdown', 'rematch', 'knitr', 'tinytex', 'xfun', 'DBI', 'blob', 'clipr', 'BH', 'cpp11', 'cellranger', 'progress', 'fs', 'rmarkdown', 'whisker', 'selectr', 'stringi', 'broom', 'dbplyr', 'forcats', 'haven', 'hms', 'lubridate', 'modelr', 'readr', 'readxl', 'reprex', 'rvest', 'stringr', 'xml2'



In [1]:
# Loading Data
library(tidyverse)

## train
df_train <- read_csv("train_to_randomForest.csv")

df_train %>% mutate_if(is.character, function(x) {str_to_upper(x)}) -> df_train
#df_train <- df_train[ , -which(names(df_train) %in% c("FIELD_34"))]



## test
df_test <- read_csv("test_to_randomForest.csv")

df_test %>% 
  mutate_if(is.character, function(x) {str_to_upper(x)}) %>% 
  mutate(FIELD_36 = as.logical(FIELD_36)) -> df_test

ERROR: Error in library(tidyverse): there is no package called 'tidyverse'


In [2]:
# Conduct binning variables: 
library(scorecard)

# Generates optimal binning for all variables/features: 
bins_var <- woebin(df_train %>% select(-id), y = "label", no_cores = 8, positive = "label|1",check_cate_num=FALSE)


Attaching package: ‘scorecard’


The following object is masked from ‘package:tidyr’:

    replace_na




[INFO] creating woe binning ... 
[INFO] Binning on 30000 rows and 64 columns in 00:07:18 


In [3]:

# IV for variables/features: 

do.call("rbind", bins_var) %>% 
  as.data.frame() %>% 
  filter(!duplicated(variable)) %>% 
  rename(iv_var = total_iv) %>% 
  arrange(iv_var) %>% 
  mutate(variable = factor(variable, levels = variable)) -> iv_values

# Features have IV >= 0: 

iv_values %>% 
  filter(iv_var >= 0) %>% 
  pull(variable) %>% 
  as.character() -> var_IV_10


# Conduct data transformation based on IV/WoE and filter features with IV > 0.1: 

train_woe <- woebin_ply(df_train %>% select(-id), bins_var) %>% 
  as.data.frame() %>% 
  select(c("label", paste0(var_IV_10, "_", "woe")))


# Data transformation for actual test data: 


## test
test_woe <- woebin_ply(df_test %>% select(-id), bins_var) %>% 
  as.data.frame() %>% 
  select(paste0(var_IV_10, "_", "woe"))

# A function imputes NA observations: 

replace_na_categorical <- function(x, y) {
  
  y %>% 
    table() %>% 
    as.data.frame() %>% 
    arrange(-Freq) -> my_df
  
  n_obs <- sum(my_df$Freq)
  
  pop <- my_df$. %>% as.character() %>% as.numeric()
  
  set.seed(29)
  
  x[is.na(x)] <- sample(pop, sum(is.na(x)), replace = TRUE, prob = my_df$Freq)
  
  return(x)
}

# Use the function: 


## test
test_woe %>% 
  mutate(FIELD_13_woe = replace_na_categorical(FIELD_13_woe, train_woe$FIELD_13_woe)) %>% 
  mutate(maCv_woe = replace_na_categorical(maCv_woe, train_woe$maCv_woe)) %>% 
  mutate(district_woe = replace_na_categorical(district_woe, train_woe$district_woe)) %>% 
  mutate(FIELD_41_woe = replace_na_categorical(FIELD_41_woe, train_woe$FIELD_41_woe)) %>% 
  mutate(FIELD_10_woe = replace_na_categorical(FIELD_10_woe, train_woe$FIELD_10_woe)) %>% 
  mutate(FIELD_39_woe = replace_na_categorical(FIELD_39_woe, train_woe$FIELD_39_woe)) %>% 
  mutate(FIELD_11_woe = replace_na_categorical(FIELD_11_woe, train_woe$FIELD_11_woe)) %>%
  #mutate(FIELD_36_woe = replace_na_categorical(FIELD_36_woe, train_woe$FIELD_36_woe)) %>%
  mutate(FIELD_9_woe = replace_na_categorical(FIELD_9_woe, train_woe$FIELD_9_woe)) %>% 
  mutate(FIELD_7_woe = replace_na_categorical(FIELD_7_woe, train_woe$FIELD_7_woe)) %>% 
  mutate(FIELD_12_woe = replace_na_categorical(FIELD_12_woe, train_woe$FIELD_12_woe)) -> test_woe_imputed

#======================================================
# Attempt 4: Default Random Forest with Scaled Data
#======================================================

# For convinience, convert binary target variable to factor: 

train_woe %>% 
  mutate(label = case_when(label == 1 ~ "Bad", TRUE ~ "Good")) %>% 
  mutate(label = as.factor(label)) -> df_forGBM 



# Scale our data: 

## train
df_forGBM %>% 
  mutate_if(is.numeric, function(x) {(x - min(x)) / (max(x) - min(x))}) -> df_forGBM_Scaled
names(df_forGBM_Scaled)



## test
test_woe_imputed %>% 
  mutate_if(is.numeric, function(x) {(x - min(x)) / (max(x) - min(x))}) -> df_test_Scaled








[INFO] converting into woe values ... 
[INFO] Woe transformating on 30000 rows and 63 columns in 00:00:10 
[INFO] converting into woe values ... 


[1] "label"                "FIELD_49_woe"         "FIELD_33_woe"        
 [4] "FIELD_48_woe"         "FIELD_34_woe"         "FIELD_2_woe"         
 [7] "FIELD_47_woe"         "FIELD_1_woe"          "FIELD_4_woe"         
[10] "FIELD_43_woe"         "FIELD_37_woe"         "FIELD_21_woe"        
[13] "FIELD_23_woe"         "FIELD_24_woe"         "FIELD_27_woe"        
[16] "FIELD_28_woe"         "FIELD_38_woe"         "FIELD_42_woe"        
[19] "FIELD_3_RESIDUAL_woe" "FIELD_35_woe"         "FIELD_32_woe"        
[22] "FIELD_45_woe"         "FIELD_16_woe"         "FIELD_44_woe"        
[25] "FIELD_40_woe"         "FIELD_36_woe"         "FIELD_46_woe"        
[28] "FIELD_41_woe"         "age_woe"              "FIELD_10_woe"        
[31] "FIELD_8_woe"          "FIELD_11_woe"         "FIELD_5_woe"         
[34] "FIELD_39_woe"         "FIELD_3_woe"          "FIELD_7_len_woe"     
[37] "FIELD_50_woe"         "FIELD_54_woe"         "FIELD_14_woe"        
[40] "FIELD_6_woe"          "FIELD_53_woe"         "FIELD_57_woe"        
[43] "FIELD_15_woe"         "FIELD_52_woe"         "FIELD_56_woe"        
[46] "FIELD_26_woe"         "FIELD_18_woe"         "FIELD_9_woe"         
[49] "FIELD_29_woe"         "FIELD_12_woe"         "FIELD_20_woe"        
[52] "FIELD_55_woe"         "FIELD_22_woe"         "FIELD_31_woe"        
[55] "FIELD_17_woe"         "FIELD_51_woe"         "FIELD_30_woe"        
[58] "FIELD_25_woe"         "FIELD_19_woe"         "FIELD_13_woe"        
[61] "province_woe"         "maCv_woe"             "district_woe"        
[64] "FIELD_7_woe"

In [4]:

# Train Random Forest: 
library(ranger)
RF_default <- ranger(label ~ ., data = df_forGBM_Scaled, probability = TRUE)


# Use the RF Classifier for predicting PD (Probability of Default): 

pd_sub_RF <- predict(RF_default, df_test_Scaled, type = "response")

# Save results for submission: 
pd_sub_RF$predictions %>% as.data.frame() %>% pull(Bad) -> pd_sub_RF
df_sub <- data.frame(id = 30000:49999, label = pd_sub_RF)
write_csv(df_sub, "submission.csv")